Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке!

Вам понадобится много признаков. Некоторые из них у вас уже есть — это:

 * идентификатор географической зоны
 * дата и время
 * количество поездок в периоды, предшествующие прогнозируемому
 * синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
 
Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!

Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования y^T+1|T, другая для y^T+2|T и т.д.

In [1]:
import warnings
import os

import pandas as pd
import numpy as np

from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import  LinearRegression
from datetime import datetime
from itertools import product
from tqdm import tqdm 

warnings.filterwarnings('ignore')
PATH_TO_DATA = 'data'

In [2]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'), index_col=0, parse_dates=[0])
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_05.csv'), index_col=0, parse_dates=[0])
kaggle_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_06.csv'), index_col=0, parse_dates=[0])


concat_df = pd.concat((train_df, test_df, kaggle_df))
print(concat_df.shape)
concat_df.head()

(4368, 102)


,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-01-01 00:00:00,80.0,144.0,50.0,77.0,319.0,402.0,531.0,617.0,846.0,267.0,...,12.0,0.0,2.0,44.0,5.0,41.0,4.0,70.0,7.0,66.0
2016-01-01 01:00:00,91.0,211.0,49.0,134.0,404.0,420.0,370.0,453.0,594.0,224.0,...,29.0,0.0,5.0,2.0,2.0,4.0,0.0,47.0,1.0,29.0
2016-01-01 02:00:00,90.0,146.0,23.0,110.0,393.0,425.0,313.0,366.0,377.0,138.0,...,47.0,0.0,3.0,0.0,4.0,0.0,0.0,69.0,1.0,14.0
2016-01-01 03:00:00,32.0,87.0,16.0,62.0,252.0,399.0,324.0,309.0,327.0,166.0,...,46.0,0.0,2.0,4.0,5.0,1.0,0.0,21.0,0.0,9.0
2016-01-01 04:00:00,24.0,43.0,10.0,53.0,145.0,254.0,264.0,333.0,318.0,145.0,...,43.0,0.0,0.0,1.0,1.0,0.0,0.0,26.0,1.0,6.0


1\. Для каждой из шести задач прогнозирования y^T+i|T,i=1,…,6 сформируйте выборки. Откликом будет yT+i при всевозможных значениях T, а признаки можно использовать следующие:

 * идентификатор географической зоны — категориальный
 * год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так
 * синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
 * сами значения прогнозов ARIMA y^ARIMAT+i|T
 * количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
 * количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
 * суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
 
Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании y^T+i|T,i=1,…,6 вы можете учитывать только значения y до момента времени T включительно.

In [3]:
def get_features(df, K = 20):
    length = df.shape[0] + 1
    features = pd.DataFrame()
    for i in range(1,K+1):
        features['sin_%d' % i] = [ np.sin(t  * 2. * i * np.pi / 168.) for t in np.arange(1, length)]
        features['cos_%d' % i] = [ np.cos(t  * 2. * i * np.pi / 168.) for t in np.arange(1, length)]

    features[['h%d' % i for i in range(0, 24)]] = pd.get_dummies(df.index.hour)
    features[['dw%d' % i for i in range(0, 7)]] = pd.get_dummies(df.index.weekday)
    features[['dm%d' % i for i in range(0, 31)]] = pd.get_dummies(df.index.day)
    features['is_weekend'] = (df.index.weekday > 4).astype(float)
    
    return features

In [4]:
feature_df = get_features(concat_df)
print(feature_df.shape)
feature_df.head()

(4368, 103)


,sin_1,cos_1,sin_2,cos_2,sin_3,cos_3,sin_4,cos_4,sin_5,cos_5,...,dm22,dm23,dm24,dm25,dm26,dm27,dm28,dm29,dm30,is_weekend
0,0.037391,0.999301,0.074730,0.997204,0.111964,0.993712,0.149042,0.988831,0.185912,0.982566,...,0,0,0,0,0,0,0,0,0,0.0
1,0.074730,0.997204,0.149042,0.988831,0.222521,0.974928,0.294755,0.955573,0.365341,0.930874,...,0,0,0,0,0,0,0,0,0,0.0
2,0.111964,0.993712,0.222521,0.974928,0.330279,0.943883,0.433884,0.900969,0.532032,0.846724,...,0,0,0,0,0,0,0,0,0,0.0
3,0.149042,0.988831,0.294755,0.955573,0.433884,0.900969,0.563320,0.826239,0.680173,0.733052,...,0,0,0,0,0,0,0,0,0,0.0
4,0.185912,0.982566,0.365341,0.930874,0.532032,0.846724,0.680173,0.733052,0.804598,0.593820,...,0,0,0,0,0,0,0,0,0,0.0


In [5]:
def rolling_window(a, window):
    a = np.concatenate((np.zeros(window), a), axis=0)
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)[:-1]

In [6]:
values = [rolling_window(concat_df[c], 5) for c in train_df.columns]
previous_rodes = pd.Series(values, index=train_df.columns)

2\. Разбейте каждую из шести выборок на три части:

 * обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
 * тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
 * итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016

In [7]:
models = {}
test_prediction_df = pd.DataFrame()
kaggle_prediction_df = pd.DataFrame()

train_size = train_df.shape[0]
kaggle_size = kaggle_df.shape[0]

3\. Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:

 * допускала попарные взаимодействия между признаками
 * была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)

In [8]:
for c in tqdm(train_df.columns):
    X = np.concatenate((feature_df[:train_size].values, previous_rodes[c][:train_size]), axis=1)
    y = train_df[c]
    
    X_test = np.concatenate((feature_df[train_size:-kaggle_size].values, previous_rodes[c][train_size:-kaggle_size]), axis=1)
    X_kaggle = np.concatenate((feature_df[-kaggle_size + 7:].values, previous_rodes[c][-kaggle_size  + 7:]), axis=1)
    
    model = LinearRegression().fit(X, y)
    models[c] = model
    
    test_prediction_df[c] = model.predict(X_test)
    kaggle_prediction_df[c] = model.predict(X_kaggle)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [00:02<00:00, 35.82it/s]


4\. Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

$$Q_{june} = {1 \over R * 715 * 6} +\sum_{r=1}^{R} \sum_{2016.04.30\ 23:00}^{2016.05.31\ 17:00}\sum_{i=1}^{6} |\hat{y}_{T|T+i} - y^r_{T+i}|$$

Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу Q, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

In [9]:
error = 0
size_of_predictions = 6 * 739 * test_prediction_df.shape[1]

for c in tqdm(test_prediction_df.columns):
    error += sum(sum(abs(rolling_window(test_prediction_df[c], 6) - rolling_window(test_df[c], 6)))) / size_of_predictions
    
error

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [00:00<00:00, 741.24it/s]


17.427576908755093

Да, результат получился заметно лучше чем на прошлой неделе.

5\. Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

6\. Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

In [10]:
index = []
values = []

start_period = june_start_period = datetime(2016, 5, 31, 17, 0, 0)
dates = [start_period + pd.DateOffset(hours=i) for i in range(721)]

for c in tqdm(kaggle_prediction_df.columns):
    t = rolling_window(kaggle_prediction_df[c], 6)
    for idx, x in enumerate(t):
        for off, i in enumerate(x):
            tm = dates[idx + off]
            index.append('{}_2016-{:02}-{:02}_{}_{}'.format(c, tm.month, tm.day, tm.hour, 6 - off))
            values.append(int(i))
            
result_df = pd.DataFrame({'y': values}, index=index)
result_df.index.name = 'id'
result_df.to_csv(os.path.join(PATH_TO_DATA, 'result.csv'))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [00:01<00:00, 67.76it/s]
